<img src="https://i.imgur.com/6U6q5jQ.png"/>

# Data Organization


Let me get the data on dengue from [Peru](https://www.datosabiertos.gob.pe/dataset/vigilancia-epidemiol%C3%B3gica-de-dengue):

In [63]:
import pandas as pd
linkData="https://github.com/SocialAnalytics-StrategicIntelligence/OrganizeExploreAndQuery/raw/main/dataFiles/dengue_ok.pkl"
dengue = pd.read_pickle(linkData)
dengue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398943 entries, 0 to 398942
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   departamento  398943 non-null  object        
 1   provincia     398943 non-null  object        
 2   distrito      398943 non-null  object        
 3   ano           398943 non-null  int64         
 4   semana        398943 non-null  int64         
 5   sexo          398943 non-null  object        
 6   edad          398943 non-null  int64         
 7   enfermedad    398943 non-null  category      
 8   year          398931 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](1), int64(3), object(4)
memory usage: 24.7+ MB


In [3]:
# some exploration
dengue.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,ano,semana,edad,year
count,398943.00000,398943.00000,398943.00000,398931.00000
mean,2015.06170,22.61685,29.97476,.5f
min,2000.00000,1.00000,0.00000,.5f
25%,2011.00000,11.00000,15.00000,.5f
50%,2016.00000,19.00000,27.00000,.5f
75%,2020.00000,34.00000,42.00000,.5f
max,2022.00000,53.00000,106.00000,.5f
std,6.14862,14.89333,18.53260,nan


Each row is a person:

In [75]:
dengue.head()

,departamento,provincia,distrito,ano,semana,sexo,edad,enfermedad,year
0,HUANUCO,LEONCIO PRADO,LUYANDO,2000,47,M,9,SIN_SEÑALES,2000-01-01
1,HUANUCO,LEONCIO PRADO,LUYANDO,2000,40,F,18,SIN_SEÑALES,2000-01-01
2,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,48,F,32,SIN_SEÑALES,2000-01-01
3,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,37,F,40,SIN_SEÑALES,2000-01-01
4,HUANUCO,LEONCIO PRADO,MARIANO DAMASO BERAUN,2000,42,M,16,SIN_SEÑALES,2000-01-01


If we wanted to count people, creating a column of ones helps:

In [6]:
dengue=dengue.assign(case=1)
dengue.head()

,departamento,provincia,distrito,ano,semana,sexo,edad,enfermedad,year,case
0,HUANUCO,LEONCIO PRADO,LUYANDO,2000,47,M,9,SIN_SEÑALES,2000-01-01,1
1,HUANUCO,LEONCIO PRADO,LUYANDO,2000,40,F,18,SIN_SEÑALES,2000-01-01,1
2,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,48,F,32,SIN_SEÑALES,2000-01-01,1
3,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,37,F,40,SIN_SEÑALES,2000-01-01,1
4,HUANUCO,LEONCIO PRADO,MARIANO DAMASO BERAUN,2000,42,M,16,SIN_SEÑALES,2000-01-01,1


Let's start creating _data from these data_!

# Dengue by Year

## Aggregation

Having people, we need to count them by some grouping variable, in this case year (_ano_) and dengue status.


In [7]:
indexList=['ano','enfermedad']
aggregator={'edad': ['mean','median'], 'case':['sum']}
ByYear_stats=dengue.groupby(indexList,observed=True).agg(aggregator)
ByYear_stats.head(20)

edad          case
                       mean median    sum
ano  enfermedad                          
2000 SIN_SEÑALES  29.508788   27.0   4324
2001 SIN_SEÑALES  30.634282   28.0  15851
     GRAVE        31.572614   28.0    241
2002 SIN_SEÑALES  26.960178   24.0   6278
     ALARMA       12.000000   12.0      1
     GRAVE        21.928571   19.0     14
2003 SIN_SEÑALES  28.947719   27.0   2850
     GRAVE        38.000000   30.0     15
2004 SIN_SEÑALES  28.863269   26.0   7928
     GRAVE        23.794118   19.0     34
2005 SIN_SEÑALES  26.891200   24.0   5000
     GRAVE        23.533333   23.0     15
2006 SIN_SEÑALES  27.436258   25.0   3624
     GRAVE        13.500000    8.5      4
2007 SIN_SEÑALES  28.676543   26.0   5299
     GRAVE        34.285714   30.0     35
2008 SIN_SEÑALES  28.941008   26.0  10493
     GRAVE        27.342857   22.0     35
2009 SIN_SEÑALES  30.180729   28.0  11271
     GRAVE        17.727273   13.0     11

In [8]:
# notice hierarchy: multindex
ByYear_stats.columns

MultiIndex([('edad',   'mean'),
            ('edad', 'median'),
            ('case',    'sum')],
           )

For easier manipualtion outside Python, we could flatten the index hierarchy:

In [10]:
# ok?
["_".join(name) for name in ByYear_stats.columns]

['edad_mean', 'edad_median', 'case_sum']

In [11]:
# changing
ByYear_stats.columns=["_".join(name) for name in ByYear_stats.columns]
ByYear_stats.head(20)

edad_mean  edad_median  case_sum
ano  enfermedad                                   
2000 SIN_SEÑALES  29.508788         27.0      4324
2001 SIN_SEÑALES  30.634282         28.0     15851
     GRAVE        31.572614         28.0       241
2002 SIN_SEÑALES  26.960178         24.0      6278
     ALARMA       12.000000         12.0         1
     GRAVE        21.928571         19.0        14
2003 SIN_SEÑALES  28.947719         27.0      2850
     GRAVE        38.000000         30.0        15
2004 SIN_SEÑALES  28.863269         26.0      7928
     GRAVE        23.794118         19.0        34
2005 SIN_SEÑALES  26.891200         24.0      5000
     GRAVE        23.533333         23.0        15
2006 SIN_SEÑALES  27.436258         25.0      3624
     GRAVE        13.500000          8.5         4
2007 SIN_SEÑALES  28.676543         26.0      5299
     GRAVE        34.285714         30.0        35
2008 SIN_SEÑALES  28.941008         26.0     10493
     GRAVE        27.342857         22.0        35
2009 SIN_SEÑALES  30.180729         28.0     11271
     GRAVE        17.727273         13.0        11

In [12]:
# final look:
ByYear_stats.reset_index(drop=False,inplace=True)
ByYear_stats.head(20)

,ano,enfermedad,edad_mean,edad_median,case_sum
0,2000,SIN_SEÑALES,29.508788,27.0,4324
1,2001,SIN_SEÑALES,30.634282,28.0,15851
2,2001,GRAVE,31.572614,28.0,241
3,2002,SIN_SEÑALES,26.960178,24.0,6278
4,2002,ALARMA,12.000000,12.0,1
5,2002,GRAVE,21.928571,19.0,14
6,2003,SIN_SEÑALES,28.947719,27.0,2850
7,2003,GRAVE,38.000000,30.0,15
8,2004,SIN_SEÑALES,28.863269,26.0,7928
9,2004,GRAVE,23.794118,19.0,34


Notice a particular data type:

In [14]:
ByYear_stats.enfermedad.dtype

CategoricalDtype(categories=['SIN_SEÑALES', 'ALARMA', 'GRAVE'], ordered=True, categories_dtype=object)

Saving to CSV will erase that _dtype_ attribute. Then, use pickle file format: 

In [254]:
ByYear_stats.to_pickle('dataFiles/ByYear_stats.pkl') # this can be read in R.

## Reshaping

Notice the variables are in three columns: **edad_mean** /	**edad_median** / 	**case_sum**. We could reshape those columns to a long format: 

In [19]:
theVarsAsIndex=['ano','enfermedad']

# stacking  and resetting index
ByYear_LongStats=ByYear_stats.set_index(theVarsAsIndex).stack().reset_index()

#result
ByYear_LongStats

,ano,enfermedad,level_2,0
0,2000,SIN_SEÑALES,edad_mean,29.508788
1,2000,SIN_SEÑALES,edad_median,27.000000
2,2000,SIN_SEÑALES,case_sum,4324.000000
3,2001,SIN_SEÑALES,edad_mean,30.634282
4,2001,SIN_SEÑALES,edad_median,28.000000
...,...,...,...,...
172,2022,ALARMA,edad_median,25.000000
173,2022,ALARMA,case_sum,7370.000000
174,2022,GRAVE,edad_mean,35.146226
175,2022,GRAVE,edad_median,32.000000


In [20]:
# just renaming
ByYear_LongStats.rename(columns={'level_2':'statsName',0:'statsValue'},inplace=True)
ByYear_LongStats

,ano,enfermedad,statsName,statsValue
0,2000,SIN_SEÑALES,edad_mean,29.508788
1,2000,SIN_SEÑALES,edad_median,27.000000
2,2000,SIN_SEÑALES,case_sum,4324.000000
3,2001,SIN_SEÑALES,edad_mean,30.634282
4,2001,SIN_SEÑALES,edad_median,28.000000
...,...,...,...,...
172,2022,ALARMA,edad_median,25.000000
173,2022,ALARMA,case_sum,7370.000000
174,2022,GRAVE,edad_mean,35.146226
175,2022,GRAVE,edad_median,32.000000


In [21]:
# still ordinal?
ByYear_LongStats.enfermedad.dtype

CategoricalDtype(categories=['SIN_SEÑALES', 'ALARMA', 'GRAVE'], ordered=True, categories_dtype=object)

In [22]:
ByYear_LongStats.to_pickle('dataFiles/ByYear_LongStats.pkl')

# Dengue by Location (Region vs Province)

## Aggregating

We can redo the previous process, adding _departamento_ and _province_: 

In [27]:
indexList=['ano','departamento','provincia','enfermedad']
aggregator={'case':['sum']}
ByYearPlace=dengue.groupby(indexList,observed=True).agg(aggregator)
ByYearPlace

case
                                             sum
ano  departamento provincia     enfermedad      
2000 AMAZONAS     BAGUA         SIN_SEÑALES  215
                  UTCUBAMBA     SIN_SEÑALES   58
     CAJAMARCA    CUTERVO       SIN_SEÑALES    2
                  JAEN          SIN_SEÑALES   16
     HUANUCO      LEONCIO PRADO SIN_SEÑALES   29
...                                          ...
2022 UCAYALI      PADRE ABAD    SIN_SEÑALES  412
                                ALARMA        87
                                GRAVE          2
                  PURUS         SIN_SEÑALES    1
                                ALARMA         1

[2300 rows x 1 columns]

Before flatteinign the output data frame in long format, you could create a wide shape:

## Long to wide

In [28]:
ByYearPlace.unstack()

case             
                                           sum             
enfermedad                         SIN_SEÑALES ALARMA GRAVE
ano  departamento provincia                                
2000 AMAZONAS     BAGUA                  215.0    NaN   NaN
                  UTCUBAMBA               58.0    NaN   NaN
     CAJAMARCA    CUTERVO                  2.0    NaN   NaN
                  JAEN                    16.0    NaN   NaN
     HUANUCO      LEONCIO PRADO           29.0    NaN   NaN
...                                        ...    ...   ...
2022 TUMBES       ZARUMILLA               89.0    5.0   NaN
     UCAYALI      ATALAYA                542.0   92.0   2.0
                  CORONEL PORTILLO      2680.0  499.0  23.0
                  PADRE ABAD             412.0   87.0   2.0
                  PURUS                    1.0    1.0   NaN

[1310 rows x 3 columns]

In [29]:
ByYearPlace_wide=ByYearPlace.unstack().reset_index()
ByYearPlace_wide

ano departamento         provincia        case             
                                                        sum             
enfermedad                                      SIN_SEÑALES ALARMA GRAVE
0           2000     AMAZONAS             BAGUA       215.0    NaN   NaN
1           2000     AMAZONAS         UTCUBAMBA        58.0    NaN   NaN
2           2000    CAJAMARCA           CUTERVO         2.0    NaN   NaN
3           2000    CAJAMARCA              JAEN        16.0    NaN   NaN
4           2000      HUANUCO     LEONCIO PRADO        29.0    NaN   NaN
...          ...          ...               ...         ...    ...   ...
1305        2022       TUMBES         ZARUMILLA        89.0    5.0   NaN
1306        2022      UCAYALI           ATALAYA       542.0   92.0   2.0
1307        2022      UCAYALI  CORONEL PORTILLO      2680.0  499.0  23.0
1308        2022      UCAYALI        PADRE ABAD       412.0   87.0   2.0
1309        2022      UCAYALI             PURUS         1.0    1.0   NaN

[1310 rows x 6 columns]

In [30]:
# no missing
ByYearPlace_wide.fillna(0,inplace=True)
ByYearPlace_wide

ano departamento         provincia        case             
                                                        sum             
enfermedad                                      SIN_SEÑALES ALARMA GRAVE
0           2000     AMAZONAS             BAGUA       215.0    0.0   0.0
1           2000     AMAZONAS         UTCUBAMBA        58.0    0.0   0.0
2           2000    CAJAMARCA           CUTERVO         2.0    0.0   0.0
3           2000    CAJAMARCA              JAEN        16.0    0.0   0.0
4           2000      HUANUCO     LEONCIO PRADO        29.0    0.0   0.0
...          ...          ...               ...         ...    ...   ...
1305        2022       TUMBES         ZARUMILLA        89.0    5.0   0.0
1306        2022      UCAYALI           ATALAYA       542.0   92.0   2.0
1307        2022      UCAYALI  CORONEL PORTILLO      2680.0  499.0  23.0
1308        2022      UCAYALI        PADRE ABAD       412.0   87.0   2.0
1309        2022      UCAYALI             PURUS         1.0    1.0   0.0

[1310 rows x 6 columns]

In [31]:
ByYearPlace_wide.columns

MultiIndex([(         'ano',    '',            ''),
            ('departamento',    '',            ''),
            (   'provincia',    '',            ''),
            (        'case', 'sum', 'SIN_SEÑALES'),
            (        'case', 'sum',      'ALARMA'),
            (        'case', 'sum',       'GRAVE')],
           names=[None, None, 'enfermedad'])

In [32]:
["_".join(names) if names[1]!='' else names[0] for names in ByYearPlace_wide.columns]

['ano',
 'departamento',
 'provincia',
 'case_sum_SIN_SEÑALES',
 'case_sum_ALARMA',
 'case_sum_GRAVE']

In [33]:
ByYearPlace_wide.columns=["_".join(names) if names[1]!='' else names[0] for names in ByYearPlace_wide.columns]
ByYearPlace_wide

,ano,departamento,provincia,case_sum_SIN_SEÑALES,case_sum_ALARMA,case_sum_GRAVE
0,2000,AMAZONAS,BAGUA,215.0,0.0,0.0
1,2000,AMAZONAS,UTCUBAMBA,58.0,0.0,0.0
2,2000,CAJAMARCA,CUTERVO,2.0,0.0,0.0
3,2000,CAJAMARCA,JAEN,16.0,0.0,0.0
4,2000,HUANUCO,LEONCIO PRADO,29.0,0.0,0.0
...,...,...,...,...,...,...
1305,2022,TUMBES,ZARUMILLA,89.0,5.0,0.0
1306,2022,UCAYALI,ATALAYA,542.0,92.0,2.0
1307,2022,UCAYALI,CORONEL PORTILLO,2680.0,499.0,23.0
1308,2022,UCAYALI,PADRE ABAD,412.0,87.0,2.0


What about finding the _provincia_ most affected in a _departamento_?

In [35]:
where = ByYearPlace_wide.groupby(['ano','departamento'])['case_sum_ALARMA'].idxmax()
worst_prov_year = ByYearPlace_wide.loc[where].reset_index(drop=True)
worst_prov_year

,ano,departamento,provincia,case_sum_SIN_SEÑALES,case_sum_ALARMA,case_sum_GRAVE
0,2000,AMAZONAS,BAGUA,215.0,0.0,0.0
1,2000,CAJAMARCA,CUTERVO,2.0,0.0,0.0
2,2000,HUANUCO,LEONCIO PRADO,29.0,0.0,0.0
3,2000,JUNIN,CHANCHAMAYO,4.0,0.0,0.0
4,2000,LA LIBERTAD,TRUJILLO,894.0,0.0,0.0
...,...,...,...,...,...,...
366,2022,PIURA,PIURA,3471.0,667.0,27.0
367,2022,PUNO,CARABAYA,25.0,0.0,0.0
368,2022,SAN MARTIN,SAN MARTIN,770.0,350.0,6.0
369,2022,TUMBES,TUMBES,515.0,28.0,0.0


In [36]:
# worst provinces
len(worst_prov_year.provincia.value_counts())

59

In [37]:
# worst >0
len(worst_prov_year[worst_prov_year.case_sum_ALARMA>0].provincia.value_counts())

43

## Filtering

Lets' filter some rows based on what we just computed:

In [42]:
worst_ProvYear_alarma=worst_prov_year[worst_prov_year.case_sum_ALARMA>0].loc[:,['departamento','provincia']]
worst_ProvYear_alarma.reset_index(drop=True,inplace=True)
worst_ProvYear_alarma

,departamento,provincia
0,LORETO,MAYNAS
1,JUNIN,SATIPO
2,LORETO,MAYNAS
3,MADRE DE DIOS,TAMBOPATA
4,PIURA,PIURA
...,...,...
198,PASCO,OXAPAMPA
199,PIURA,PIURA
200,SAN MARTIN,SAN MARTIN
201,TUMBES,TUMBES


In [43]:
worst_ProvYear_alarma['case']=1
worst_ProvYear_alarma

,departamento,provincia,case
0,LORETO,MAYNAS,1
1,JUNIN,SATIPO,1
2,LORETO,MAYNAS,1
3,MADRE DE DIOS,TAMBOPATA,1
4,PIURA,PIURA,1
...,...,...,...
198,PASCO,OXAPAMPA,1
199,PIURA,PIURA,1
200,SAN MARTIN,SAN MARTIN,1
201,TUMBES,TUMBES,1


## Frequency table

With filtered data, let's create a crosstabulation:

In [44]:
indexList=['departamento','provincia']
aggregator={'case':['sum']}
worst_ProvYear_alarma_Frequency=worst_ProvYear_alarma.groupby(indexList,observed=True).agg(aggregator)
worst_ProvYear_alarma_Frequency

case
                                sum
departamento  provincia            
AMAZONAS      BAGUA               6
              UTCUBAMBA           6
ANCASH        CASMA               5
              SANTA               3
AREQUIPA      AREQUIPA            1
AYACUCHO      LA MAR              7
              SUCRE               1
CAJAMARCA     CAJAMARCA           1
              JAEN               10
CALLAO        CALLAO              2
CUSCO         LA CONVENCION       9
              QUISPICANCHI        1
HUANUCO       LEONCIO PRADO      10
              PUERTO INCA         2
ICA           CHINCHA             1
              ICA                 3
              NAZCA               1
              PALPA               2
JUNIN         CHANCHAMAYO         9
              SATIPO              4
LA LIBERTAD   ASCOPE              2
              CHEPEN              2
              TRUJILLO            6
LAMBAYEQUE    CHICLAYO            4
              FERREÑAFE           2
              LAMBAYEQUE          3
LIMA          LIMA                9
LORETO        ALTO AMAZONAS       1
              MAYNAS             13
MADRE DE DIOS TAMBOPATA          13
PASCO         OXAPAMPA           11
PIURA         HUANCABAMBA         1
              PIURA              10
              SULLANA             1
              TALARA              1
PUNO          CARABAYA            2
SAN MARTIN    HUALLAGA            1
              MARISCAL CACERES    2
              SAN MARTIN          8
              TOCACHE             2
TUMBES        TUMBES             12
              ZARUMILLA           1
UCAYALI       CORONEL PORTILLO   12

In [45]:
worst_ProvYear_alarma_Frequency.reset_index()

,departamento,provincia,case
,,,sum
0,AMAZONAS,BAGUA,6
1,AMAZONAS,UTCUBAMBA,6
2,ANCASH,CASMA,5
3,ANCASH,SANTA,3
4,AREQUIPA,AREQUIPA,1
5,AYACUCHO,LA MAR,7
6,AYACUCHO,SUCRE,1
7,CAJAMARCA,CAJAMARCA,1
8,CAJAMARCA,JAEN,10


In [46]:
worst_ProvYear_alarma_Frequency.columns=['case']
worst_ProvYear_alarma_Frequency.reset_index(inplace=True)
worst_ProvYear_alarma_Frequency

,departamento,provincia,case
0,AMAZONAS,BAGUA,6
1,AMAZONAS,UTCUBAMBA,6
2,ANCASH,CASMA,5
3,ANCASH,SANTA,3
4,AREQUIPA,AREQUIPA,1
5,AYACUCHO,LA MAR,7
6,AYACUCHO,SUCRE,1
7,CAJAMARCA,CAJAMARCA,1
8,CAJAMARCA,JAEN,10
9,CALLAO,CALLAO,2


Saving the frequencies:

In [47]:
worst_ProvYear_alarma_Frequency.to_csv('dataFiles/worst_ProvYear_alarma_Frequency.csv',index=False)

# Dengue by Location and Year

## Aggregating

Let's check a previous data frame:

In [9]:
ByYearPlace_wide

,ano,departamento,provincia,case_sum_SIN_SEÑALES,case_sum_ALARMA,case_sum_GRAVE
0,2000,AMAZONAS,BAGUA,215.0,0.0,0.0
1,2000,AMAZONAS,UTCUBAMBA,58.0,0.0,0.0
2,2000,CAJAMARCA,CUTERVO,2.0,0.0,0.0
3,2000,CAJAMARCA,JAEN,16.0,0.0,0.0
4,2000,HUANUCO,LEONCIO PRADO,29.0,0.0,0.0
...,...,...,...,...,...,...
1305,2022,TUMBES,ZARUMILLA,89.0,5.0,0.0
1306,2022,UCAYALI,ATALAYA,542.0,92.0,2.0
1307,2022,UCAYALI,CORONEL PORTILLO,2680.0,499.0,23.0
1308,2022,UCAYALI,PADRE ABAD,412.0,87.0,2.0


This time, I want two variables:

In [48]:
indexList=['ano','departamento']
aggregator={'case_sum_SIN_SEÑALES':['sum'],'case_sum_ALARMA':['sum']}
ByYearPlace=ByYearPlace_wide.groupby(indexList,observed=True).agg(aggregator)
ByYearPlace.columns=['sum_SIN_SEÑALES','sum_ALARMA']
ByYearPlace.reset_index(inplace=True)
ByYearPlace

,ano,departamento,sum_SIN_SEÑALES,sum_ALARMA
0,2000,AMAZONAS,273.0,0.0
1,2000,CAJAMARCA,18.0,0.0
2,2000,HUANUCO,29.0,0.0
3,2000,JUNIN,7.0,0.0
4,2000,LA LIBERTAD,894.0,0.0
...,...,...,...,...
366,2022,PIURA,9296.0,1361.0
367,2022,PUNO,25.0,0.0
368,2022,SAN MARTIN,3229.0,907.0
369,2022,TUMBES,656.0,36.0


## Creating information

I will create a new variable:

In [49]:
ByYearPlace['rateAlarma']=(ByYearPlace['sum_ALARMA']/ByYearPlace['sum_SIN_SEÑALES'])
ByYearPlace['rateAlarma'].describe()

/Users/JoseManuel/opt/anaconda3/envs/SpatialDS_311_pandas2/lib/python3.11/site-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


count    371.000000
mean            inf
std             NaN
min        0.000000
25%        0.000000
50%        0.018216
75%        0.134195
max             inf
Name: rateAlarma, dtype: float64

We got _inf_ values:

In [52]:
import numpy as np
ByYearPlace[np.isinf(ByYearPlace.rateAlarma)]

,ano,departamento,sum_SIN_SEÑALES,sum_ALARMA,rateAlarma
186,2013,AYACUCHO,0.0,1.0,inf


We need to make a decision. I did this:

In [53]:
ByYearPlace.loc[186,'rateAlarma']=1
ByYearPlace.drop(columns=['sum_SIN_SEÑALES','sum_ALARMA'],inplace=True)
ByYearPlace['rateAlarma'].describe()

count    371.000000
mean       0.111113
std        0.202358
min        0.000000
25%        0.000000
50%        0.018216
75%        0.134195
max        1.583333
Name: rateAlarma, dtype: float64

### Discretizing

Sometimes you need a numerical variable as an ordinal variable: 

In [56]:
edges=[-1, .1, .25, .5,.75,1,2]
theLabels=["less10%","10-25%","25-50","51-75%","75-100%","above100%"]
ByYearPlace["rateAlarma.cut"]=pd.cut(ByYearPlace['rateAlarma'], include_lowest=True,
                                     bins=edges, 
                                     labels=theLabels,
                                     ordered=True)

In [57]:
# we have
ByYearPlace

,ano,departamento,rateAlarma,rateAlarma.cut
0,2000,AMAZONAS,0.000000,less10%
1,2000,CAJAMARCA,0.000000,less10%
2,2000,HUANUCO,0.000000,less10%
3,2000,JUNIN,0.000000,less10%
4,2000,LA LIBERTAD,0.000000,less10%
...,...,...,...,...
366,2022,PIURA,0.146407,10-25%
367,2022,PUNO,0.000000,less10%
368,2022,SAN MARTIN,0.280892,25-50
369,2022,TUMBES,0.054878,less10%


We could check the yearly behavior:

In [58]:
ByYearPlace.groupby('ano').describe()

rateAlarma                                                              \
          count      mean       std       min       25%       50%       75%   
ano                                                                           
2000       11.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2001       14.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2002       15.0  0.000030  0.000116  0.000000  0.000000  0.000000  0.000000   
2003       12.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2004       13.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2005       16.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2006       14.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2007       15.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2008       14.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2009       15.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2010       14.0  0.008610  0.018810  0.000000  0.000000  0.000000  0.005710   
2011       15.0  0.099278  0.141586  0.000000  0.005208  0.043902  0.083672   
2012       16.0  0.150327  0.121813  0.000000  0.069813  0.122838  0.189647   
2013       18.0  0.375294  0.350709  0.000000  0.094267  0.274368  0.541558   
2014       16.0  0.285056  0.406160  0.000000  0.030391  0.097071  0.421336   
2015       18.0  0.314695  0.305296  0.000000  0.091302  0.275115  0.416486   
2016       18.0  0.190514  0.157750  0.007685  0.050252  0.158646  0.325561   
2017       19.0  0.151913  0.146412  0.000000  0.024195  0.087728  0.303617   
2018       19.0  0.142461  0.178510  0.000000  0.000000  0.059140  0.244079   
2019       19.0  0.127213  0.120950  0.000000  0.036147  0.083333  0.189534   
2020       19.0  0.169504  0.233548  0.005703  0.029017  0.093544  0.196144   
2021       21.0  0.135615  0.229660  0.000000  0.031120  0.072072  0.166667   
2022       20.0  0.135949  0.096298  0.000000  0.064062  0.127371  0.187478   

                
           max  
ano             
2000  0.000000  
2001  0.000000  
2002  0.000451  
2003  0.000000  
2004  0.000000  
2005  0.000000  
2006  0.000000  
2007  0.000000  
2008  0.000000  
2009  0.000000  
2010  0.069204  
2011  0.398804  
2012  0.487755  
2013  1.000000  
2014  1.583333  
2015  1.337931  
2016  0.509953  
2017  0.378788  
2018  0.519616  
2019  0.372638  
2020  1.000000  
2021  1.076923  
2022  0.381858

Let's do some **filtering**:

In [59]:
ByYearPlace=ByYearPlace[ByYearPlace.ano>=2012]
ByYearPlace.reset_index(drop=True,inplace=True)
ByYearPlace

,ano,departamento,rateAlarma,rateAlarma.cut
0,2012,AMAZONAS,0.126273,10-25%
1,2012,ANCASH,0.021111,less10%
2,2012,CAJAMARCA,0.177215,10-25%
3,2012,HUANUCO,0.309237,25-50
4,2012,JUNIN,0.061286,less10%
...,...,...,...,...
198,2022,PIURA,0.146407,10-25%
199,2022,PUNO,0.000000,less10%
200,2022,SAN MARTIN,0.280892,25-50
201,2022,TUMBES,0.054878,less10%


In [60]:
ByYearPlace.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   ano             203 non-null    int64   
 1   departamento    203 non-null    object  
 2   rateAlarma      203 non-null    float64 
 3   rateAlarma.cut  203 non-null    category
dtypes: category(1), float64(1), int64(1), object(1)
memory usage: 5.3+ KB


In [24]:
# the categort should be exported as pickle

ByYearPlace.to_pickle("dataFiles/ByYearPlace.pkl")